#Functions used in my simulation

all of the plotting code and simulation data is stored in a text file called gmfunc.py

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.integrate import odeint
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import display
from IPython.display import Image,HTML,SVG

:0: FutureWarning: IPython widgets are experimental and may change in the future.


The code below was developed to be able to find solutions to these second order differential equations

$$ \ddot{\mathbf{r}} = -\gamma \left\{ \frac{M}{r^3}\mathbf{r} -\frac{S}{\rho^3}\boldsymbol{\rho} + \frac{S}{R^3}\boldsymbol\Re \right\} $$

$$ \ddot{\boldsymbol\Re} = -\gamma \frac{M+S}{R^3}\boldsymbol\Re$$

Since they are second order differential equations, 4 components were needed for each vector. The gamma I have calculated is with Solar units for mass, $10^{8}$ years for time, and kiloparsecs for distance

In [2]:

gamma=4.4936287*(10**(-8))
def derivs(rvec,t,M,S):
    rx=rvec[0]
    ry=rvec[1]
    vx=rvec[2]
    vy=rvec[3]
    Rx=rvec[4]
    Ry=rvec[5]
    Vx=rvec[6]
    Vy=rvec[7]
    
    R=np.sqrt(Rx**2+Ry**2)
    r=np.sqrt(rx**2+ry**2)
    rho = np.sqrt((Rx-rx)**2 + (Ry-ry)**2)
    
    drx=vx
    dry=vy
    dvx=-gamma*((M/(r**3))*rx+(S/(R**3))*Rx-(S/(rho**3))*(Rx-rx))
    dvy=-gamma*((M/(r**3))*ry+(S/(R**3))*Ry-(S/(rho**3))*(Ry-ry))
    dRx=Vx
    dRy=Vy
    dVx=-gamma*((M+S)/(R**3))*Rx
    dVy=-gamma*((M+S)/(R**3))*Ry
    return np.array([drx,dry,dvx,dvy,dRx,dRy,dVx,dVy])

This code was made to create the intial position and velocity conditions for all of the 120 stars in the simulation. The rmin input is the closest the galaxy S gets to galaxy M, and the s value determins the direction of spin.

For counter clockise spin s=1, and for clockwise spin s=-1

In [3]:
def create_ic_stars(rmin,s):
    ics = []
    
    # loop
    for n in range(0,36,1):
        vr=np.sqrt(4493.6287/(0.6*rmin))
        x=(0.6*rmin*np.cos(np.pi*2*n/36))
        y=(0.6*rmin*np.sin(np.pi*2*n/36))
        vx=(vr*np.sin(np.pi*2*n/36+np.pi))
        vy=(vr*np.cos(np.pi*2*n/36))
        ic = np.array([x,y,vx,vy])
        ics.append(ic)
    for n in range(0,30,1):
        vr=np.sqrt(4493.6287/(0.5*rmin))
        x=(0.5*rmin*np.cos(np.pi*2*n/30))
        y=(0.5*rmin*np.sin(np.pi*2*n/30))
        vx=(vr*np.sin(np.pi*2*n/30+np.pi))
        vy=(vr*np.cos(np.pi*2*n/30))
        ic = np.array([x,y,vx,vy])
        ics.append(ic)
    for n in range(0,24,1):
        vr=np.sqrt(4493.6287/(0.4*rmin))
        x=(0.4*rmin*np.cos(np.pi*2*n/24))
        y=(0.4*rmin*np.sin(np.pi*2*n/24))
        vx=(vr*np.sin(np.pi*2*n/24+np.pi))
        vy=(vr*np.cos(np.pi*2*n/24))
        ic = np.array([x,y,vx,vy])
        ics.append(ic)
    for n in range(0,18,1):
        vr=np.sqrt(4493.6287/(0.3*rmin))
        x=(0.3*rmin*np.cos(np.pi*2*n/18))
        y=(0.3*rmin*np.sin(np.pi*2*n/18))
        vx=(vr*np.sin(np.pi*2*n/18+np.pi))
        vy=(vr*np.cos(np.pi*2*n/18))
        ic = np.array([x,y,vx,vy])
        ics.append(ic)
    for n in range(0,12,1):
        vr=np.sqrt(4493.6287/(0.2*rmin))
        x=(0.2*rmin*np.cos(np.pi*2*n/12))
        y=(0.2*rmin*np.sin(np.pi*2*n/12))
        vx=(vr*np.sin(np.pi*2*n/12+np.pi))
        vy=(vr*np.cos(np.pi*2*n/12))
        ic = np.array([x,y,s*vx,s*vy])
        ics.append(ic)

    return ics


This code below is used to create the initial conditions for the S galaxy.

Sy is the starting height of galaxy S relative to M (kiloparsecs), rmin is the closest you want S to get to M (kiloparsecs), and M and S are inputs for the mass of each galaxy in solar units.  

In [5]:
def create_ic_gala(Sy,rmin,M,S):
    Sx=-(1/(4*rmin))*Sy**2+rmin
    vr=np.sqrt(2*gamma*(M+S)/(np.sqrt(Sx**2+Sy**2)))
    angle=np.arctan(2*rmin/Sy)
    Vx=vr*np.cos(angle)
    Vy=-vr*np.sin(angle)
    return np.array([Sx,Sy,Vx,Vy])


The solve_one_star function takes in the intial conditions of both the galaxy and whatever star you pick, and combines these inputes using np.hstack. This set of intial conditions along with my derivs functions from earlier are then put into odeint which solves for the position and velocities of both the star and galaxy S at the time values selected. Those time values are deteremined by the tmax and ntimes input, which splits your max time(tmax) chosen intoa specific number of division(ntimes), all with equal step sizes.

In [ ]:
def solve_one_star(ic_star, ic_gala, tmax, ntimes,M,S):
    t = np.linspace(0,tmax,ntimes)
    ic = np.hstack([ic_star, ic_gala])
    soln = odeint(derivs, ic, t,(M,S))
    return soln

The solve_all_stars functions uses the solve_star_function, but it also iterates through all of the intial conditions provided instead of only one. The initial conditons I used

In [ ]:
def solve_all_stars(ic_stars, ic_gala, tmax, ntimes,M,S):
    solns = []
    for ic_star in ic_stars:
        soln = solve_one_star(ic_star, ic_gala, tmax, ntimes,M,S)
        solns.append(soln)
    return solns
